In [3]:
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# Step 1: Unzip the dataset
zip_path = 'Delivery-Starter.zip'  # Update if path is different
extract_to = 'delivery_data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# Step 2: Load the CSV file (assuming there is only one CSV inside)
import os
for file in os.listdir(extract_to):
    if file.endswith('.csv'):
        data_path = os.path.join(extract_to, file)
        break

df = pd.read_csv(data_path)

# Step 3: Inspect data
print(df.head())
print(df.info())
print(df.describe())

# Step 4: Visualize target distribution (assuming 'delivery_time' is the target)
sns.histplot(df['delivery_time'], kde=True)
plt.title("Distribution of Delivery Time")
plt.show()

# Step 5: Feature selection (adjust based on actual columns)
# Example: remove non-numeric or unneeded columns
X = df.drop(columns=['delivery_time', 'order_id'], errors='ignore')  # Replace with actual columns
y = df['delivery_time']

# Optional: handle categorical variables
X = pd.get_dummies(X, drop_first=True)

# Step 6: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Step 8: Evaluate model
y_pred = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))


FileNotFoundError: [Errno 2] No such file or directory: 'Delivery-Starter.zip'

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

# Step 1: Handle missing values (basic)
df = df.dropna()

# Step 2: Convert datetime columns if any (example)
# df['pickup_time'] = pd.to_datetime(df['pickup_time'])
# df['drop_time'] = pd.to_datetime(df['drop_time'])
# df['duration'] = (df['drop_time'] - df['pickup_time']).dt.total_seconds() / 60

# Step 3: Correlation matrix
plt.figure(figsize=(10, 6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

# Step 4: Feature engineering (adjust as per data)
X = df.drop(columns=['delivery_time', 'order_id'], errors='ignore')  # adjust as needed
y = df['delivery_time']

X = pd.get_dummies(X, drop_first=True)

# Optional: Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 6: Train Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Step 7: Evaluation
y_pred = rf.predict(X_test)
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R² Score:", r2_score(y_test, y_pred))

# Step 8: Feature Importance
importances = pd.Series(rf.feature_importances_, index=X.columns)
importances.sort_values().tail(10).plot(kind='barh')
plt.title("Top 10 Important Features")
plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder

# Load data
df = pd.read_csv('Delivery.csv')  # Update with correct file path

# Parse datetime columns
df['pickup_time'] = pd.to_datetime(df['pickup_time'])
df['drop_time'] = pd.to_datetime(df['drop_time'])

# Feature engineering
df['delivery_duration'] = (df['drop_time'] - df['pickup_time']).dt.total_seconds() / 60  # in minutes
df['hour_of_day'] = df['pickup_time'].dt.hour
df['day_of_week'] = df['pickup_time'].dt.dayofweek

# Drop rows with missing or negative durations
df = df[df['delivery_duration'] > 0].dropna()

# Encode categorical features
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Select features and target
features = ['distance', 'hour_of_day', 'day_of_week', 'vendor_id']  # Add more as needed
X = df[features]
y = df['delivery_duration']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model with hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5]
}

rf = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_

# Predict and evaluate
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Params: {grid_search.best_params_}")
print(f"MAE: {mae:.2f} mins")
print(f"R² Score: {r2:.2f}")
